In [3]:
from __future__ import print_function
import ipywidgets as widgets
from traitlets import Unicode, validate, CBytes, Float

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = Unicode('Hello World!').tag(sync=True)
    duration = Float().tag(sync=True)
    interval = Float().tag(sync=True)
    midi = CBytes().tag(sync=True)

In [4]:
%%javascript
require.config({
    paths: {
        webaudiofontplayer: '../files/webaudiofont/npm/dist/WebAudioFontPlayer',
        midifile: '../files/webaudiofont/examples/MIDIFile'
    }
});

<IPython.core.display.Javascript object>

In [5]:
%%javascript
require.undef('hello');

define('hello', ["@jupyter-widgets/base", 
                 'webaudiofontplayer', 
                 'midifile'], function(widgets) {

    var audioContext = null;
    var player = null;
    var reverberator = null;
    var songStart = 0;
    var input = null;
    var currentSongTime = 0;
    var nextStepTime = 0;
    var nextPositionTime = 0;
    var loadedsong = null;
    var stepDuration = 44;

    var HelloView = widgets.DOMWidgetView.extend({

        render: function() {
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
            this.midi_changed();
            this.model.on('change:midi', this.midi_changed, this);
        },

        value_changed: function() {
            this.el.textContent = this.model.get('value');
        },
        
        midi_changed: function() {
            var midi = this.model.get('midi').buffer;
            var midiFile = new MIDIFile( midi);
            loadedsong = midiFile.parseSong();
            this.startLoad( loadedsong)
            this.updateModel()
        },

        updateModel: function(){
            this.model.set('duration', loadedsong.duration)
            this.model.set('interval', stepDuration)
        },
        
        startLoad: function(song) {
            console.log(song);
            var AudioContextFunc = window.AudioContext || window.webkitAudioContext;
            audioContext = new AudioContextFunc();
            player = new WebAudioFontPlayer();
            reverberator = player.createReverberator(audioContext);
            reverberator.output.connect(audioContext.destination);
            input = reverberator.input;
            for (var i = 0; i < song.tracks.length; i++) {
                var nn = player.loader.findInstrument(song.tracks[i].program);
                var info = player.loader.instrumentInfo(nn);
                song.tracks[i].info = info;
                song.tracks[i].id = nn;
                player.loader.startLoad(audioContext, info.url, info.variable);
            }
            for (var i = 0; i < song.beats.length; i++) {
                var nn = player.loader.findDrum(song.beats[i].n);
                var info = player.loader.drumInfo(nn);
                song.beats[i].info = info;
                song.beats[i].id = nn;
                player.loader.startLoad(audioContext, info.url, info.variable);
            }
            player.loader.waitLoad(function () {
            });
        },

    });

    return {
        HelloView : HelloView
    };
});

<IPython.core.display.Javascript object>

In [11]:
with open( './webaudiofont/examples/midi/123.mid', 'rb') as f:
    w = HelloWidget( midi = f.read())

play = widgets.Play( max=100, step=3.5, interval=2000)
slider = widgets.FloatSlider()
#widgets.jslink((w, 'duration'), (play, 'max'))
#widgets.jslink((w, 'duration'), (slider, 'max'))
#widgets.jslink((w, 'interval'), (play, 'interval'))
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.VBox([play, slider, w])

In [7]:
w.value = 'test'